In [2]:
import os
import numpy as np
# import working directory to check functions
os.chdir('/Users/schiend/Desktop/DOHERTY/R-workspace/cecelia/inst')

# config
import py.config_utils as cfg

%load_ext autoreload
%autoreload 2

In [3]:
base_dir = '/Volumes/USER_data/Dominik/Experiments/stomics/10x/Xenium_FFPE_Human_Breast_Cancer_Rep1_outs'
transcripts_path = os.path.join(base_dir, 'transcripts.csv.gz')
im_path = os.path.join(base_dir, 'morphology_mip.ome.tif')
ts_zarr_path = os.path.join(base_dir, 'ts.zarr')

In [5]:
# read transcript data
import pandas as pd

ts_data = pd.read_csv(transcripts_path, compression = 'gzip', error_bad_lines = False)
ts_data

/var/folders/rb/m9wptf8x58x_82t7lg081kfs1fj7kv/T/ipykernel_1743/935456629.py:4: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  ts_data = pd.read_csv(transcripts_path, compression = 'gzip', error_bad_lines = False)


,transcript_id,cell_id,overlaps_nucleus,feature_name,x_location,y_location,z_location,qv
0,281474976710656,565,0,SEC11C,4.395842,328.66647,12.019493,18.662480
1,281474976710657,540,0,NegControlCodeword_0502,5.074415,236.96484,7.608511,18.634956
2,281474976710658,562,0,SEC11C,4.702023,322.79715,12.289083,18.662480
3,281474976710659,271,0,DAPK3,4.906601,581.42865,11.222615,20.821745
4,281474976710660,291,0,TCIM,5.660699,720.85175,9.265523,18.017488
...,...,...,...,...,...,...,...,...
42638078,281805689407068,-1,0,HOXD8,5218.000500,5295.51950,30.560934,20.061079
42638079,281805689407071,135717,1,LUM,5218.804700,4792.92800,38.440426,21.389278
42638080,281805689407078,-1,0,LUM,5217.405000,5425.38600,30.519602,21.960102
42638081,281805689407083,-1,0,NARS,5220.155300,4839.70400,32.180700,25.970398


In [6]:
max_values = ts_data['qv'].max()

In [71]:
# get data and meta data
im_data, _ = zarr_utils.open_as_zarr(im_path)
omexml = ome_xml_utils.parse_meta(im_path)
dim_utils = DimUtils(omexml)
dim_utils.calc_image_dimensions(im_data[0].shape)
pixel_sizes = dim_utils.im_physical_sizes()

(25778, 35416)

In [94]:
len(im_data)

8

In [75]:
## convert to zarr image
import math

# build up image dimensions
dim_cols = ['y_location', 'x_location']
x_min, y_min = ts_data[dim_cols].min(axis = 0)
x_max, y_max = ts_data[dim_cols].max(axis = 0)

im_dim = [
    0, 0,
    dim_utils.im_dim[0],
    dim_utils.im_dim[1]
]
im_dim

[0, 0, 25778, 35416]

In [76]:
channel_names = ts_data['feature_name'].unique()

# filter blanks
channel_names = [x for x in channel_names if x.startswith(('BLANK', 'NegControl')) is False]

# DEBUG only use defined ones for now
channel_names = [
    # B
    'BANK1', 'CD79A', 'MS4A1',
    # T
    'CCL5', 'CD4', 'CD8A', 'CXCR4', 'CYTIP',
    'IL7R', 'LTB', 'TRAC',
    # Mphage
    #'APOC1', 'C15orf48', 'C1QA', 'C1QC', 'CD14', 
    #'CD163', 'CD68', 'FGL2', 'ITGAX', 'MMP12',
    # DC
    #'CCR7', 'CD83', 'IL3RA', 'LILRA4', 'PLD4',
    # Stroma
    #'ALDH1A3', 'GJB2', 'LUM', 'MMP2', 'POSTN', 'SFRP4'
]

num_channels = len(channel_names)

In [77]:
import shutil
import bioformats
import zarr

import py.ome_xml_utils as ome_xml_utils
import py.zarr_utils as zarr_utils
from py.dim_utils import DimUtils

In [78]:
from scipy.sparse import coo_array
import skimage.filters
import skimage.morphology

In [79]:
# add to napari
from py.napari_utils import NapariUtils

napari_utils = NapariUtils()
napari_utils.viewer = None
napari_utils.open_viewer()

In [80]:
napari_utils.open_image(
    im_path, use_channel_axis = False, as_dask = True
)

[25778, 35416]


In [81]:
# create zarr
zarr_shape = (
    num_channels,
    im_dim[2] - im_dim[0],
    im_dim[3] - im_dim[1],
    #im_dim[5] - im_dim[2]
)
  
seq_image = zarr.open(
    ts_zarr_path,
    mode = 'w',
    shape = zarr_shape,
    chunks = (1, 512, 512),
    dtype = np.float16
)

In [86]:
import random

filter_value = 1

shutil.rmtree(ts_zarr_path)

# go through and create images
for i, x in enumerate(channel_names):
    print(x)
    
    y1 = ts_data.loc[ts_data['feature_name'] == x]
    
    # create sparse matrix
    row  = y1[dim_cols[0]].values / pixel_sizes['y']
    col  = y1[dim_cols[1]].values / pixel_sizes['x']
    data = y1['qv'].values / max_values

    y2 = coo_array((data, (row, col)), shape = zarr_shape[1:3]).toarray()

    # copy in with gaussian
    seq_image[i, :, :] = skimage.filters.gaussian(y2, filter_value)
    
    #seq_image[i, :, :] = skimage.filters.median(
    #    seq_image[i, :, :], skimage.morphology.disk(filter_value))
    
# generate multiscales 
nscales = 5

# TODO is there a more elegant way to do this .. ?
if nscales > 1:
    multiscales_file_path = ts_zarr_path + ".multiscales"

zarr_utils.create_multiscales(
  seq_image, multiscales_file_path,
  x_idx = 1, y_idx = 2,
  nscales = nscales
)

# remove previous and rename multiscales
shutil.rmtree(ts_zarr_path)
os.rename(multiscales_file_path, ts_zarr_path)

# build metadata
o = bioformats.omexml.OMEXML()

# TODO anything else?
o.image().Pixels.channel_count = zarr_shape[0]
o.image().Pixels.set_SizeC(zarr_shape[0])
o.image().Pixels.set_SizeX(zarr_shape[2])
o.image().Pixels.set_SizeY(zarr_shape[1])
o.image().Pixels.set_PhysicalSizeX(1)
o.image().Pixels.set_PhysicalSizeY(1)
o.image().Pixels.set_PhysicalSizeXUnit('um')
o.image().Pixels.set_PhysicalSizeYUnit('um')
o.image().Pixels.set_PixelType('uint16')

#for i, x in enumerate(channel_names[0:num_channels]):
for i, x in enumerate(channel_names):
    o.image().Pixels.Channel(i).Name = x

# add metadata
ome_xml_utils.write_ome_xml(ts_zarr_path, o)

BANK1



KeyboardInterrupt



In [87]:
napari_utils.open_image(
    ts_zarr_path, use_channel_axis = True, as_dask = True, 
    channel_names = channel_names, visible = False,
    contrast_limits = [0, 1]
)

PathNotFoundError: nothing found at path ''

In [50]:
ome_xml_utils.parse_from_tiff(im_path)

'<OME xmlns="http://www.openmicroscopy.org/Schemas/OME/2016-06" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" Creator="tifffile.py 2021.4.8" UUID="urn:uuid:a73aa686-7468-11ed-89c9-06acdb970e37" xsi:schemaLocation="http://www.openmicroscopy.org/Schemas/OME/2016-06 http://www.openmicroscopy.org/Schemas/OME/2016-06/ome.xsd">\n    <Plate ID="Plate:1" WellOriginX="-0.0" WellOriginXUnit="µm" WellOriginY="-0.0" WellOriginYUnit="µm" />\n    <Instrument ID="Instrument:1">\n        <Microscope Manufacturer="10x Genomics" Model="Xenium" />\n    </Instrument>\n    <Image ID="Image:0" Name="Image0">\n        <InstrumentRef ID="Instrument:1" />\n        <Pixels DimensionOrder="XYZCT" ID="Pixels:0" SizeC="1" SizeT="1" SizeX="35416" SizeY="25778" SizeZ="1" Type="uint16" PhysicalSizeX="0.2125" PhysicalSizeY="0.2125">\n            <Channel ID="Channel:0:0" Color="-1" Name="DAPI" SamplesPerPixel="1" />\n            <TiffData PlaneCount="1" />\n        </Pixels>\n    </Image>\n</OME>'

In [ ]:
# read in OME XML from first image sequence
import tifffile
import py.zarr_utils as zarr_utils
import py.ome_xml_utils as ome_xml_utils
from py.dim_utils import DimUtils

im = tifffile.TiffFile(im_path)
x_array = im.asarray()
omexml = ome_xml_utils.parse_meta(im_path)
dim_utils = DimUtils(omexml)
dim_utils.calc_image_dimensions(x_array.shape)

In [ ]:
# split y axis
# TODO this is specific to Thorlabs 3P output
split_size = int(dim_utils.dim_val('Y')/dim_utils.dim_val('X'))

In [ ]:
# DEBUG take first x slices
debug_slices = 12
debug_start_slice = 48

In [ ]:
# split array along Y
x_split = np.array_split(x_array, split_size, axis = dim_utils.dim_idx('Y'))

In [ ]:
x_split = [x[debug_start_slice:(debug_start_slice + debug_slices), :, :, :] for x in x_split]

In [ ]:
stack_dim = 'Z'
skip_tiles = 1
nscales = 1
physical_stack_scale = 2

In [ ]:
# get or create stack axis
stack_dim_idx = dim_utils.dim_idx(stack_dim)
stack_array = False

# check whether to create this dimension
if stack_dim_idx is None:
    stack_dim_idx = dim_utils.default_order.index(stack_dim)
    stack_array = True

In [ ]:
import math

In [ ]:
"""
scale intensity
exp[𝑚𝑛(Δ𝑧/ls − ln(𝑠))] 
m - multiphoton number
n - plane index
s - relative incident laser power ratio between planes
d𝑧 - spacing between planes
ls - mean free path
"""
def reverb_intensity_scale(m, n, s, dz, ls, x = None, scale = None):
    # get y 
    y = 1/math.exp(m * n * (dz/ls - math.log(s)))
    
    # scale?
    if scale is not None:
        y = y/scale
    
    if x is not None:
        return (x * y).astype(x.dtype)
    else:
        return y

In [ ]:
m = 3
s = 2
dz = 26
ls = 200

In [ ]:
reverb_intensity_scale(m, len(x_split_shift[::(skip_tiles + 1)]), s, dz, ls)

In [ ]:
# create shifts to create image
shifts = np.array([
    [14.74, -2.18],
    [14.74, -2.18],
    [13.41, 16.31],
    [13.41, 16.31],
    [8.92, -4.23],
    [8.92, -4.23]
])

In [ ]:
# generate slices for images
shift_slices_target = [[slice(None) for _ in x_array.shape] for _ in range(len(shifts))]
shift_slices_source = [[slice(None) for _ in x_array.shape] for _ in range(len(shifts))]
dim_val = dim_utils.dim_val('X')

for i, shift in enumerate(shifts):
    # create slices for source and target
    if shift[0] > 0:
        shift_slices_target[i][dim_utils.dim_idx('Y')] = slice(round(shift[0]), dim_val, 1)
        shift_slices_source[i][dim_utils.dim_idx('Y')] = slice(0, dim_val - round(shift[0]), 1)
    else:
        shift_slices_target[i][dim_utils.dim_idx('Y')] = slice(0, dim_val + round(shift[0]), 1)
        shift_slices_source[i][dim_utils.dim_idx('Y')] = slice(-round(shift[0]), dim_val, 1)
        
    if shift[1] > 0:
        shift_slices_target[i][dim_utils.dim_idx('X')] = slice(round(shift[1]), dim_val, 1)
        shift_slices_source[i][dim_utils.dim_idx('X')] = slice(0, dim_val - round(shift[1]), 1)
    else:
        shift_slices_target[i][dim_utils.dim_idx('X')] = slice(0, dim_val + round(shift[1]), 1)
        shift_slices_source[i][dim_utils.dim_idx('X')] = slice(-round(shift[1]), dim_val, 1)
    
shift_slices_target = [tuple(x) for x in shift_slices_target]
shift_slices_source = [tuple(x) for x in shift_slices_source]

In [ ]:
x_split_shift = x_split[:2]

# generate new image
for i, x in enumerate(x_split[2:]):
    # zero image
    zero_im = np.zeros_like(x)
    
    # copy image
    zero_im[shift_slices_target[i]] = x[shift_slices_source[i]]
    # zero_im[shift_slices_source[i]] = x[shift_slices_target[i]]
    
    x_split_shift.append(zero_im)

In [ ]:
 if stack_array is True:
    x_new = np.stack(
        [reverb_intensity_scale(x, m, n + 1, s, dz, ls) \
         for n, x in enumerate(x_split[::(skip_tiles + 1)])],
        axis = stack_dim_idx
    )
else:
    # get max reverb scale
    max_scale = reverb_intensity_scale(m, len(x_split_shift[::(skip_tiles + 1)]), s, dz, ls)
    
    # contact with shift slices
    x_new_list = [reverb_intensity_scale(m, n + 1, s, dz, ls, x = x, scale = max_scale) \
         for n, x in enumerate(x_split_shift[::(skip_tiles + 1)])]
    x_new_list.reverse()
    
    x_new = np.concatenate(x_new_list, axis = stack_dim_idx)

In [ ]:
# use median filter
from skimage.filters.rank import median
from skimage.morphology import ball

In [ ]:
im_slices = [slice(None) for _ in range(len(x_new.shape))]

for i in range(dim_utils.dim_val('C')):
    im_slices = list(im_slices)
    im_slices[dim_utils.dim_idx('C')] = i
    im_slices = tuple(im_slices)
    
    x_new[im_slices] = median(x_new[im_slices], ball(3))

In [ ]:
zarr_utils.create_multiscales(x_new, zarr_path, nscales = nscales)

In [ ]:
# create shape dict for new image
shape_dict = {'Y': int(dim_utils.dim_val('Y')/split_size)}
shape_dict[stack_dim.upper()] = x_new.shape[stack_dim_idx]
scale_dict = dict()
scale_dict[stack_dim.upper()] = physical_stack_scale

# change image dimensions in xml
omexml_new = ome_xml_utils.set_im_size_with_dict(omexml, shape_dict)
omexml_new = ome_xml_utils.set_physical_size_with_dict(omexml_new, scale_dict)

# add metadata
ome_xml_utils.write_ome_xml(zarr_path, omexml_new)

In [ ]:
from py.napari_utils import NapariUtils

napari_utils = NapariUtils()
napari_utils.viewer = None
napari_utils.open_viewer()

In [ ]:
channel_names = [
    'A', 'B', 'C', 'D'
]

In [ ]:
napari_utils.open_image(
    zarr_path,
    use_channel_axis = True, as_dask = True,
    channel_names = channel_names
)

In [ ]:
from napari_animation import AnimationWidget
animation_widget = AnimationWidget(napari_utils.viewer)
napari_utils.viewer.window.add_dock_widget(animation_widget, area='right')